# <p style="font-size:200%; text-align:center; margin: 0"> BoomBikes Prediction🚴‍♀️📈 </p>

# Table of Contents

* [1. Introduction](#intro)
    * [1.1. Problem Statement](#problem)
    * [1.2. Business Goal](#goal)
    * [1.3. Dataset Overview](#dataoverview)
* [2. Data Understanding and Processing](#data)
    * [2.1. Loading the Data](#loading)
    * [2.2. Initial Exploration and Dataset Characteristics](#ieda)
    * [2.3. Data Preprocessing](#dataprep)
* [3. Exploratory Data Analysis](#eda)
    * [3.1. Univariate Analysis](#u_analysis)
    * [3.2. Bivariate Analysis](#b_analysis)
    * [3.3. Multivariate Analysis](#m_analysis)
* [4. Feature Engineering](#feature)
* [5. Model Building and Feature Selection](#model)
    * [5.1. Feature Selection Using Recursive Feature Elimination (RFE) ](#rfe)
    * [5.2. Model Development and Comparison](#models)
    * [5.3. Selecting the Best Model](#best_model)
    * [5.4. Final Model](#final_model)
    * [5.5. Model Evaluation](#model_evaluation)
        * [5.5.1. Residual Analysis](#residual_analysis)
        * [5.5.2. Predicting for Test Data](#predicting_test_data)
* [6. Conclusions and Recommendations](#conclusions)

# 1. Introduction <a id="intro"></a>

## 1.1. Problem Statement <a id="problem"></a>
A bike-sharing system is a service in which bikes are made available for shared use to individuals on a short-term basis for a price or free. BoomBikes, a US-based bike-sharing provider, has experienced a significant decline in revenue due to the ongoing COVID-19 pandemic. To address this challenge, the company aims to develop a strategic business plan to increase revenue once the lockdown ends and the economy stabilizes.

The company seeks to understand the factors affecting the demand for shared bikes in the American market. Specifically, BoomBikes wants to know:
- Which variables are significant in predicting the demand for shared bikes.
- How well those variables describe the bike demands.

## 1.2. Business Goal <a id="goal"></a>
The goal is to model the demand for shared bikes using the available independent variables. This model will help the management understand the demand dynamics and adjust their business strategy accordingly to meet customer expectations and gain a competitive edge in the market.

## 1.3. Dataset Overview <a id="dataoverview"></a>
- instant: record index
- dteday : date
- season : season (1:spring, 2:summer, 3:fall, 4:winter)
- yr : year (0: 2018, 1:2019)
- mnth : month ( 1 to 12)
- holiday : weather day is a holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
- weathersit : 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : temperature in Celsius
- atemp: feeling temperature in Celsius
- hum: humidity
- windspeed: wind speed
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both STM)casual and registered

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import sklearn
import statsmodels.api as sm
import scipy.stats as stats 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

# 2. Data Understanding and Processing <a id="data"></a>

## 2.1. Loading the Data<a id="loading"></a>

In [ ]:
bike_df = pd.read_csv('day.csv')
bike_df.head()

## 2.2. Initial Exploration and Dataset Characteristics<a id="ieda"></a>

In [ ]:
bike_df.shape

### Checking for Null Values

In [ ]:
bike_df.info()

In [ ]:
# Checking Statistical Description 
bike_df.describe().T

## 2.3. Data Preprocessing <a id="dataprep"></a>

In [ ]:
bike_df = bike_df.set_index('instant')

In [ ]:
# Renaming Some column Names
bike_df.rename(columns={'yr': 'year', 'mnth': 'month', 'hum':'humidity', 'cnt': 'count', 'dteday':'date'}, inplace=True)

In [ ]:
# Assigning values to categorical values
bike_df['season'] = bike_df['season'].map({1:'spring', 2:'summer', 3:'fall', 4:'winter'})
bike_df['weathersit'] = bike_df['weathersit'].map({1:'Clear',2:'Mist',3:'Light_Snow_Rain',4:'Heavy_Snow_Rain'})
bike_df['month'] = bike_df['month'].replace((1,2,3,4,5,6,7,8,9,10,11,12),('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'))
bike_df['weekday'] = bike_df['weekday'].map({1: 'Wed', 2: 'Thurs', 3: 'Fri', 4: 'Sat', 5: 'Sun', 6: 'Mon', 0: 'Tues'})

In [ ]:
bike_df.head()

### Outliers Handling

In [ ]:
columns = ['humidity', 'temp', 'atemp', 'windspeed']

# Setting up the subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(6, 6))

# Formatting Figure
fig.suptitle('Boxplots of Bike Rentals by Various Factors', fontsize=16, fontweight='bold')

# Flatten the axes array for easy iteration
axes = axes.flatten()

for i, var in enumerate(columns):
    sns.boxplot(ax=axes[i], x=var, data=bike_df, palette='muted')
    axes[i].set_xlabel(var, fontsize=10, fontweight='bold')

# Turn off the last empty subplot
if len(columns) < len(axes):
    axes[-1].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96]) 
plt.show()

It seems that there are no such outliers are present which needs to remove.

# 3. Exploratory Data Analysis <a id="eda"></a>

- Exploring the data is essential for gaining insights and understanding its underlying patterns.

- One of the most significant insights from this dataset is the trend in bike share occurrences over time.

## 3.1. Univariate Analysis <a id="u_analysis"></a>

#### Numerical Variables

In [ ]:
columns = ['count', 'humidity', 'temp', 'atemp', 'windspeed']
# Setting up the subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 12))

# Formatting Figure
fig.suptitle('Histograms of Bike Rentals by Various Factors', fontsize=16, fontweight='bold')

# Flatten the axes array for easy iteration
axes = axes.flatten()
for ax, var in zip(axes, columns):
    sns.histplot(ax=ax, x=var, data=bike_df, palette='muted', kde=True)
    ax.set_xlabel(var, fontsize=10, fontweight='bold')
    ax.set_ylabel('Count', fontsize=10, fontweight='bold')
# Turn off the last empty subplot
if len(columns) < len(axes):
    axes[-1].axis('off')


plt.tight_layout(rect=[0, 0, 1, 0.96]) 
plt.show()

#### Categorical variable

In [ ]:
columns = ['season', 'weathersit', 'holiday', 'workingday', 'weekday', 'month']
# Setting up the subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 12))

# Formatting Figure
fig.suptitle('Countplot of Bike Rentals by Various Factors', fontsize=16, fontweight='bold')

# Flatten the axes array for easy iteration
axes = axes.flatten()
for ax, var in zip(axes, columns):
    sns.countplot(ax=ax, x = bike_df[var], width=0.5, palette='Set2')
    ax.set_xlabel(var, fontsize=10, fontweight='bold')
    ax.set_ylabel('Frequency', fontsize=10, fontweight='bold')

plt.tight_layout(rect=[0, 0, 1, 0.96]) 
plt.show()

## 3.2. Bivariate analysis <a id="b_analysis"></a>

In [ ]:
columns = ['temp', 'atemp', 'humidity', 'windspeed']
# Setting up the subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))

# Formatting Figure
fig.suptitle('Scatter Plots of Bike Rentals Against Weather Variables', fontsize=16, fontweight='bold')

# Flatten the axes array for easy iteration
axes = axes.flatten()
for ax, var in zip(axes, columns):
    sns.scatterplot(ax=ax, x=var, y='count', data=bike_df, palette='muted')
    ax.set_xlabel(var, fontsize=10, fontweight='bold')
    ax.set_ylabel('Count', fontsize=10, fontweight='bold')

plt.tight_layout(rect=[0, 0, 1, 0.96]) 
plt.show()

***Inference***:
- Temperature (both actual and apparent) seems to be a significant factor in bike rentals, with warmer conditions leading to higher rental counts.
- Humidity and windspeed appear to have less pronounced effects, though high humidity and high windspeed might slightly deter rentals.

In [ ]:
columns = ['temp', 'atemp', 'humidity', 'windspeed']
# Setting up the subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))

# Formatting Figure
fig.suptitle('Relationship Between Bike Rentals and Weather Variables Across Seasons', fontsize=16, fontweight='bold')

# Flatten the axes array for easy iteration
axes = axes.flatten()
for ax, var in zip(axes, columns):
    sns.scatterplot(ax=ax, x=var, y='count', data=bike_df, hue='season',  palette='muted')
    ax.set_xlabel(var, fontsize=10, fontweight='bold')
    ax.set_ylabel('Count', fontsize=10, fontweight='bold')

plt.tight_layout(rect=[0, 0, 1, 0.96]) 
plt.show()

In [ ]:
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.figure(figsize=(6, 4))
sns.catplot(data=bike_df, x='month', y='count', order=month_order, kind="point", hue='year', marker='o', aspect=1.5)
plt.title('Month-wise Count Trend')
plt.xlabel('Month')
plt.ylabel('Total Count')
plt.grid(True)
plt.show()

**Inference:**
- Seasonal Trend: Bike rentals peak during the warmer months (June-Sept) and drop in the colder months (January, December), indicating a strong seasonal influence on demand.

- Year-over-Year Increase: The 2019 consistently shows higher rental counts than 2018, suggesting increased popularity or improved conditions for biking.

## 3.3. Multivariate Analysis <a id="m_analysis"></a>

In [ ]:
# Pairplot of numerical columns
columns = ['temp', 'atemp', 'humidity', 'windspeed', 'count']

# Create the pair plot
sns.pairplot(bike_df[columns])

# Show the plot
plt.suptitle('Pair Plot of Numerical Columns', fontsize=16, fontweight='bold', y=1.02)
plt.show()

In [ ]:
# Correlation Matrix of Numerical Columns
columns = ['temp', 'atemp', 'humidity', 'windspeed', 'count']
cmap = sns.color_palette("viridis", as_cmap=True)
sns.heatmap(bike_df[columns].corr(), cmap=cmap, annot=True, fmt=".3f", linewidths=0.5, center=0,annot_kws={"size": 7, "color": "black"})
# Add titles and labels
plt.title('Correlation Heatmap', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)

# Show the heatmap
plt.show()

In [ ]:
bike_df.drop(['date', 'casual', 'registered'], inplace=True, axis=1)

# 4. Feature Engineering <a id="feature"></a>

In [ ]:
bike_df.columns

#### Creating Dummy Variables

In [ ]:
season_dummy = pd.get_dummies(bike_df.season, drop_first=True, dtype=int)
month_dummy = pd.get_dummies(bike_df.month, drop_first=True, dtype=int)
weather_dummy = pd.get_dummies(bike_df.weathersit, drop_first=True, dtype=int)
weekday_dummy = pd.get_dummies(bike_df.weekday, drop_first=True, dtype=int)

In [ ]:
weather_dummy

In [ ]:
df_with_dummies = pd.concat([bike_df, season_dummy, month_dummy, weather_dummy, weekday_dummy], axis=1)
df_with_dummies.drop(['season', 'month', 'weathersit', 'weekday'], axis=1, inplace=True)

In [ ]:
cmap = sns.color_palette("viridis", as_cmap=True)
plt.figure(figsize=(12, 8))

sns.heatmap(df_with_dummies.corr(), cmap=cmap, annot=True, fmt=".1f", linewidths=0.5, center=0,annot_kws={"size": 7, "color": "black"})
# Add titles and labels
plt.title('Correlation Heatmap', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)

# Show the heatmap
plt.show()

**We observed that some dummy variables are correlated with each other. To address this, we are opting to manually drop specific columns instead of relying on the drop_first parameter when creating the dummies**

In [ ]:
weather_dummy = pd.get_dummies(bike_df.weathersit, dtype=int)
weekday_dummy = pd.get_dummies(bike_df.weekday, dtype=int)


In [ ]:
bike_df = pd.concat([bike_df, season_dummy, month_dummy, weather_dummy, weekday_dummy], axis=1)
bike_df.drop(['season', 'month', 'weathersit', 'weekday'], axis=1, inplace=True)

In [ ]:
cmap = sns.color_palette("viridis", as_cmap=True)
plt.figure(figsize=(12, 8))

sns.heatmap(bike_df.corr(), cmap=cmap, annot=True, fmt=".1f", linewidths=0.5, center=0,annot_kws={"size": 7, "color": "black"})
# Add titles and labels
plt.title('Correlation Heatmap', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)

# Show the heatmap
plt.show()

In [ ]:
corr_matrix = bike_df[['count', 'Clear', 'Light_Snow_Rain', 'Mist' ]].corr()
cmap = sns.color_palette("viridis", as_cmap=True)
plt.figure(figsize=(5, 4))
sns.heatmap(corr_matrix.corr(), cmap=cmap, annot=True, fmt=".2f", linewidths=0.5, center=0,annot_kws={"size": 7, "color": "black"})
# Add titles and labels
plt.title('Correlation Heatmap', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)

# Show the heatmap
plt.show()

In [ ]:
# Dropping Mist as it has high correlation with clear weather
bike_df.drop('Mist', axis=1, inplace=True)

In [ ]:
corr_matrix = bike_df[['count', 'Fri', 'Mon', 'Sat', 'Sun', 'Thurs', 'Tues', 'Wed' ]].corr()
cmap = sns.color_palette("viridis", as_cmap=True)
plt.figure(figsize=(5, 4))
sns.heatmap(corr_matrix.corr(), cmap=cmap, annot=True, fmt=".2f", linewidths=0.5, center=0,annot_kws={"size": 7, "color": "black"})
# Add titles and labels
plt.title('Correlation Heatmap', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)

# Show the heatmap
plt.show()

In [ ]:
# Dropping Thurs as its least correlated with count
bike_df.drop('Thurs', axis=1, inplace=True)

#### Splitting the Data into Training and Testing Sets

In [ ]:
df_train, df_test = train_test_split(bike_df, train_size=0.7, test_size=0.3, random_state=100)

#### Rescaling the Features


- Normalization (Min-Max Scaling): Rescales the data to a fixed range, usually [0,1]. The formula is:
$$
X_{\text{scaled}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}}
$$

In [ ]:
scaler = MinMaxScaler()

In [ ]:
df_train.columns

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['temp', 'atemp', 'humidity', 'windspeed']
df_train[num_vars] = scaler.fit_transform(df_train[num_vars])
df_test[num_vars] = scaler.transform(df_test[num_vars])

In [ ]:
df_train.describe()

#### Dividung into X and Y sets for the model building

In [ ]:
y_train = df_train.pop('count')
X_train = df_train
y_test = df_test.pop('count')
X_test = df_test

# 5. Model Building and Feature Selection <a id="model"></a>

### Base Model

In [ ]:
def linear_reg(X_train, y_train, X_test, y_test):
    '''
    This function performs linear regression on the provided training data, evaluates the model on both training 
    and testing data, and returns the trained model.

    Parameters:
    X_train (DataFrame or array-like): The training feature set.
    y_train (Series or array-like): The target variable corresponding to the training data.
    X_test (DataFrame or array-like): The testing feature set.
    y_test (Series or array-like): The target variable corresponding to the testing data.'''
    
    X_train_lm = sm.add_constant(X_train)
    model = sm.OLS(y_train, X_train_lm).fit()
    X_test_lm = sm.add_constant(X_test)
    y_pred_train = model.predict(X_train_lm)
    y_pred_test= model.predict(X_test_lm)
    print("R2 Score of the training data:",r2_score(y_pred = y_pred_train, y_true = y_train))
    print("R2 Score of the testing data:",r2_score(y_pred = y_pred_test, y_true = y_test))
    print(model.summary())
    plt.show()
    return model

In [ ]:
def get_stats(model, X):
    # Add a constant to X (just as it was added during the model fitting)
    X_with_const = sm.add_constant(X)
    
    # Calculate p-values and VIF
    stats = pd.DataFrame(data=model.pvalues, columns=['Pvalue'])
    stats['feature'] = stats.index
    
    # Ensure VIF calculation includes the constant
    stats['VIF'] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]
    
    stats.reset_index(drop=True, inplace=True)
    stats['Pvalue'] = stats['Pvalue'].round(3)
    
    # Sort by p-value in descending order
    stats = stats.sort_values(by='Pvalue', ascending=False)
    return stats

In [ ]:
base_model = linear_reg(X_train, y_train, X_test, y_test)

In [ ]:
get_stats(base_model, X_train)

## 5.1. Feature Selection Using Recursive Feature Elimination (RFE) <a id='rfe'></a>

In [ ]:
# Performing RFE
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(estimator=lm, n_features_to_select=15)
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# Extracting RFE Selected Features
col = X_train.columns[rfe.support_]
col

In [ ]:
X_train.columns[~rfe.support_]

## 5.2. Model Development and Comparison <a id='models'></a>

***Developing a model using features selected by RFE, and refining it by removing features based on p-values and VIF to achieve the most optimized performance.***

### Model_0

In [ ]:
# Building Model Using RFE selected Features
X_train_lm = X_train[col]
X_test_lm = X_test[col]

In [ ]:
model_0 = linear_reg(X_train_lm, y_train, X_test_lm, y_test)

In [ ]:
get_stats(model_0, X_train_lm)

### Model 1

In [ ]:
# Removing the 'holiday' feature due to its p-value of 0.42 and an infinite VIF
col = ['year', 'workingday', 'temp', 'humidity', 'windspeed',
       'spring', 'winter', 'Jan', 'Jul', 'Sep', 'Clear', 'Light_Snow_Rain',
       'Mon', 'Tues']

X_train_lm = X_train[col]
X_test_lm = X_test[col]
model_1 = linear_reg(X_train_lm, y_train, X_test_lm, y_test)

In [ ]:
get_stats(model_1, X_train_lm)

### Model 2

In [ ]:
# Removing the 'Tues' feature due to its VIF of 5.7
col = ['year', 'workingday', 'temp', 'humidity', 'windspeed',
       'spring', 'winter', 'Jan', 'Jul', 'Sep', 'Clear', 'Light_Snow_Rain',
       'Mon']
X_train_lm = X_train[col]
X_test_lm = X_test[col]
model_2 = linear_reg(X_train_lm, y_train, X_test_lm, y_test)

In [ ]:
get_stats(model_2, X_train_lm)

### Model 3

In [ ]:
# Removing the 'Jan' feature
col = ['year', 'workingday', 'temp', 'humidity', 'windspeed',
       'spring', 'winter', 'Jul', 'Sep', 'Clear', 'Light_Snow_Rain',
       'Mon']
X_train_lm = X_train[col]
X_test_lm = X_test[col]
model_3 = linear_reg(X_train_lm, y_train, X_test_lm, y_test)

In [ ]:
get_stats(model_3, X_train_lm)

### Model 4

In [ ]:
# Removing 'temp' feature and trying to add 'atemp' - They both are correlated to each other so using only one of them 
col = ['year', 'workingday', 'atemp', 'humidity', 'windspeed',
       'spring', 'winter', 'Jul', 'Sep', 'Clear', 'Light_Snow_Rain',
       'Mon']
X_train_lm = X_train[col]
X_test_lm = X_test[col]
model_4 = linear_reg(X_train_lm, y_train, X_test_lm, y_test)

In [ ]:
get_stats(model_4, X_train_lm)


In [ ]:
cmap = sns.color_palette("viridis", as_cmap=True)
plt.figure(figsize=(10, 6))

sns.heatmap(X_train[col].corr(), cmap=cmap, annot=True, fmt=".1f", linewidths=0.5, center=0,annot_kws={"size": 7, "color": "black"})
# Add titles and labels
plt.title('Correlation Heatmap', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)

# Show the heatmap
plt.show()


### Model 5

In [ ]:
# Removing the 'Mon' feature
col = ['year', 'workingday', 'temp', 'humidity', 'windspeed',
       'spring', 'winter', 'Jul', 'Sep', 'Clear', 'Light_Snow_Rain']

X_train_lm = X_train[col]
X_test_lm = X_test[col]
model_5 = linear_reg(X_train_lm, y_train, X_test_lm, y_test)

In [ ]:
get_stats(model_5, X_train_lm)

### Model 6

In [ ]:
# Removing the 'Workingday' feature
col = ['year', 'temp', 'humidity', 'windspeed',
       'spring', 'winter', 'Jul', 'Sep', 'Clear', 'Light_Snow_Rain']
X_train_lm = X_train[col]
X_test_lm = X_test[col]
model_6 = linear_reg(X_train_lm, y_train, X_test_lm, y_test)

In [ ]:
get_stats(model_6, X_train_lm)

### Model 7

In [ ]:
col = ['year', 'temp', 'humidity', 'windspeed',
       'spring', 'winter', 'Jul', 'Sep', 'Light_Snow_Rain']
X_train_lm = X_train[col]
X_test_lm = X_test[col]
model_7 = linear_reg(X_train_lm, y_train, X_test_lm, y_test)

In [ ]:
get_stats(model_7, X_train_lm)

## 5.3. Selecting the Best Model <a id='best_model'></a>

In [ ]:
models = [model_1, model_2, model_3, model_4, model_5, model_6, model_7]  

# Create an empty DataFrame to store the metrics
model_comparison = pd.DataFrame(columns=['Model', 'R-squared', 'Adjusted R-squared', 'F-statistic', 'AIC', 'BIC'])

# Loop through each fitted model and extract the statistics
for i, model in enumerate(models, start=1):
    # Create a DataFrame for the current model's statistics
    model_stats = pd.DataFrame({
        'Model': [f'Model_{i}'],
        'R-squared': [model.rsquared],
        'Adjusted R-squared': [model.rsquared_adj],
        'F-statistic': [model.fvalue],
        'AIC': [model.aic],
        'BIC': [model.bic]
    })

    # Concatenate the current model's statistics with the main DataFrame
    model_comparison = pd.concat([model_comparison, model_stats], ignore_index=True)

# Display the DataFrame with results
model_comparison

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(14, 10))
fig.suptitle('Model Comparison Metrics', fontsize=16)

# Function to add values on bars
def add_values_on_bars(ax, bars):
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Plot each metric on a separate subplot and add values on bars
bars_r2 = axes[0, 0].bar(model_comparison['Model'], model_comparison['R-squared'], color='skyblue')
axes[0, 0].set_title('R-squared')
axes[0, 0].set_xlabel('Models')
axes[0, 0].set_ylabel('R-squared')
add_values_on_bars(axes[0, 0], bars_r2)

bars_adj_r2 = axes[0, 1].bar(model_comparison['Model'], model_comparison['Adjusted R-squared'], color='lightgreen')
axes[0, 1].set_title('Adjusted R-squared')
axes[0, 1].set_xlabel('Models')
axes[0, 1].set_ylabel('Adjusted R-squared')
add_values_on_bars(axes[0, 1], bars_adj_r2)

bars_fstat = axes[1, 0].bar(model_comparison['Model'], model_comparison['F-statistic'], color='salmon')
axes[1, 0].set_title('F-statistic')
axes[1, 0].set_xlabel('Models')
axes[1, 0].set_ylabel('F-statistic')
add_values_on_bars(axes[1, 0], bars_fstat)

bars_aic = axes[1, 1].bar(model_comparison['Model'], model_comparison['AIC'], color='orchid')
axes[1, 1].set_title('AIC')
axes[1, 1].set_xlabel('Models')
axes[1, 1].set_ylabel('AIC')
add_values_on_bars(axes[1, 1], bars_aic)

bars_bic = axes[2, 0].bar(model_comparison['Model'], model_comparison['BIC'], color='gold')
axes[2, 0].set_title('BIC')
axes[2, 0].set_xlabel('Models')
axes[2, 0].set_ylabel('BIC')
add_values_on_bars(axes[2, 0], bars_bic)

# Hide the last unused subplot
axes[2, 1].axis('off')

# Adjust layout
plt.tight_layout(rect=[0, 0, 1, 0.96])

# Show plot
plt.show()

**Best Model: Model_3**

***Reason for Selection***: Model_3 has one of the highest R-squared and Adjusted R-squared values, indicating strong explanatory power. It avoids the multicollinearity issue seen in Models 1 and 2, while still maintaining a favorable balance in AIC and BIC, making it a robust and reliable choice.

## 5.4. Final Model <a id='final_model'></a>

In [ ]:
# Final Model
col = ['year', 'workingday', 'temp', 'humidity', 'windspeed',
       'spring', 'winter', 'Jul', 'Sep', 'Clear', 'Light_Snow_Rain',
       'Mon']
X_train_lm = X_train[col]
X_test_lm = X_test[col]
final_model = linear_reg(X_train_lm, y_train, X_test_lm, y_test)

## 5.5. Model Evaluation <a id='model_evalution'></a>

### 5.5.1. Residual Analysis <a id='residual_analysis'></a>

- Residual analysis helps assess how well the model's predictions align with actual data, identifying any patterns or discrepancies that suggest model inaccuracies.

- It ensures that key regression assumptions (e.g., homoscedasticity, linearity) are met, which is crucial for the validity of the model's inferences.

In [ ]:
y_train_pred = final_model.predict(sm.add_constant(X_train_lm))
residuals = y_train - y_train_pred

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 12))

# 1. Homoscedasticity Check
sns.scatterplot(x=y_train_pred, y=residuals, alpha=0.5, ax=axs[0, 0])
axs[0, 0].axhline(0, color='red', linestyle='--', linewidth=1)
axs[0, 0].set_xlabel("Predicted Values")
axs[0, 0].set_ylabel("Residuals")
axs[0, 0].set_title("Homoscedasticity Check")

# 2. Normality of Residuals Check with Histogram and KDE
sns.histplot(residuals, kde=True, ax=axs[0, 1])
axs[0, 1].set_xlabel("Residuals")
axs[0, 1].set_title("Normality of Error Terms Check")

# 3. Q-Q Plot for Normality Check
stats.probplot(residuals, dist="norm", plot=axs[1, 0])
axs[1, 0].set_title("Q-Q Plot")

# 4. Multicollinearity Check
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(X_train_lm.corr(), cmap=cmap, annot=True, fmt=".2f", ax=axs[1, 1])
axs[1, 1].set_title("Multi-Collinearity Check")

# Set a common title for the entire figure
fig.suptitle('Regression Assumptions Check', fontsize=16, fontweight='bold')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to fit the title
plt.show()

### 5.5.2. Test Data Predictions <a id='predicting_test_data'></a>

In [ ]:
y_train_pred = final_model.predict(sm.add_constant(X_train_lm))
y_test_pred= final_model.predict(sm.add_constant(X_test_lm))

In [ ]:
print("R2 Score of the testing data:",r2_score(y_pred = y_test_pred, y_true = y_test))

In [ ]:
c = [i for i in range(1, 220)]  # Create a list of indices from 1 to 219
fig = plt.figure(figsize=(16, 6)) 
plt.plot(c, y_test, color="blue", linewidth=1.5, linestyle="-", label='Actual')  
plt.plot(c, y_test_pred, color="red", linewidth=1.5, linestyle="-", label='Predicted')  

fig.suptitle('Actual and Predicted', fontsize=20)
plt.xlabel('Index', fontsize=18)  
plt.ylabel('Count', fontsize=16)  
plt.show()

***Final Model Equation***


<p style="font-weight: bold; font-style: italic; background-color: #f0f0f0;">
  count = 1977.9951 + 2007.7272⋅year + 450.4622⋅workingday + 4160.0340⋅temp − 1259.2241⋅humidity − 1470.3284⋅windspeed 
  − 939.9930⋅spring + 492.2803⋅winter − 668.0764⋅Jul + 497.1687⋅Sep + 513.9474⋅Clear − 1663.1473⋅Light_Snow_Rain + 536.7014⋅Mon
</p>
        

**Interpretation:**

- **R-Squared**: The R-squared values for the training and testing sets are **0.842** and **0.820**, respectively, indicating that the model explains a substantial proportion of the variance in the bike rental count. The adjusted R-squared value is **0.838**, further supporting the model's validity.

- **Significant Predictors**:
  - All predictors have statistically significant coefficients (p-values close to zero), meaning that they contribute meaningfully to the model.
  
- **Coefficients**:
  - **Positive impact**: Variables like `year`, `workingday`, `temp`, `winter`, `Sep`, `Clear`, and `Mon` have positive coefficients, meaning that increases in these variables are associated with an increase in bike rental counts.
  - **Negative impact**: Variables like `humidity`, `windspeed`, `spring`, `Jul`, and `Light_Snow_Rain` have negative coefficients, indicating that increases in these factors are associated with a decrease in bike rental counts.

# 6. Conclusions and Recommendations <a id='conclusions'></a>

1. ***Focus on Temperature Management***
- **Insight:** Temperature significantly impacts bike rentals (**coef = 4160.0340**), indicating a preference for rentals during favorable temperatures.
- **Recommendation:** Promote rentals during optimal temperature conditions through discounts or shaded routes.

2. ***Optimize for Working Days***
- **Insight:** Rentals increase on working days (**coef = 450.4622**).
- **Recommendation:** Target promotions for non-working days and explore partnerships with companies to promote bike commuting.

3. ***Seasonal Promotions***
- **Insight:** Winter rentals increase (**coef = 492.2803**), while spring rentals decrease (**coef = -939.9930**).
- **Recommendation:** Tailor promotions based on seasonal trends, such as offering winter gear or hosting spring cycling events.

4. ***Weather-Responsive Pricing***
- **Insight:** Clear weather boosts rentals (**coef = 513.9474**), while light snow or rain decreases them (**coef = -1663.1473**).
- **Recommendation:** Implement dynamic pricing based on weather conditions.

5. ***Strategic Weekday Focus***
- **Insight:** Rentals are higher on Mondays (**coef = 536.7014**).
- **Recommendation:** Focus marketing efforts on Mondays with weekday-specific deals.

6. ***Monthly Campaigns***
- **Insight:** September shows an increase in rentals (**coef = 497.1687**), while July shows a decrease (**coef = -668.0764**).
- **Recommendation:** Leverage September's trend with marketing pushes and address the July decline with targeted campaigns.

7. ***Yearly Growth Leverage***
- **Insight:** The year 2019 positively impacts rentals (**coef = 2007.7272**).
- **Recommendation:** Continue expansion efforts to sustain growth, including marketing and exploring new markets.

8. ***Corporate and Institutional Partnership***
- **Insight:** Working days positively impact rentals.
- **Recommendation:** Partner with corporate offices, schools, and universities to promote bike rentals as a sustainable transport option.